## Setup

MLP IMAGE CLASSFICATION


In [3]:
pip install -U tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 21.8 MB/s eta 0:00:00


In [4]:

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Prepare the data

In [79]:
import tensorflow as tf
import pathlib

# Define the paths to the training and testing data folders
train_data_path = "/content/drive/MyDrive/Azymer/AlzimerSVMModel/PreprocessSeg"
test_data_path = "/content/drive/MyDrive/Azymer/AlzimerSVMModel/PreprocessSeg"

# Use TensorFlow's image data generator to load the images and labels from the folders
train_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_data = train_data_generator.flow_from_directory(directory=train_data_path,
                                                      target_size=(128, 128),
                                                      batch_size=32,
                                                      class_mode='categorical',
                                                      shuffle=True)

test_data = test_data_generator.flow_from_directory(directory=test_data_path,
                                                    target_size=(128,128),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    shuffle=False)

# Print the shapes of the data and labels
print("Training data shape:", train_data.image_shape)
print("Training labels shape:", train_data.classes.shape)
print("Testing data shape:", test_data.image_shape)
print("Testing labels shape:", test_data.classes.shape)

# Assign the data and labels to variables x_train, y_train, x_test, and y_test
x_train, y_train = train_data.next()
x_test, y_test = test_data.next()

Found 6400 images belonging to 4 classes.
Found 6400 images belonging to 4 classes.
Training data shape: (128, 128, 3)
Training labels shape: (6400,)
Testing data shape: (128, 128, 3)
Testing labels shape: (6400,)


In [80]:
val_split = 0.1

val_indices = int(len(x_train) * val_split)
new_x_train, new_y_train = x_train[val_indices:], y_train[val_indices:]
x_val, y_val = x_train[:val_indices], y_train[:val_indices]

print(f"Training data samples: {len(new_x_train)}")
print(f"Validation data samples: {len(x_val)}")
print(f"Test data samples: {len(x_test)}")

Training data samples: 29
Validation data samples: 3
Test data samples: 32


In [81]:
import tensorflow as tf
import numpy as np

# Assume y_train and y_test have a shape of [?, 4]
# Convert them to integer labels
y_train_int = np.argmax(y_train, axis=1)
y_test_int = np.argmax(y_test, axis=1)

# Now you can convert them to one-hot encoded vectors with num_classes=4
y_train = tf.keras.utils.to_categorical(y_train_int, num_classes=4)
y_test = tf.keras.utils.to_categorical(y_test_int, num_classes=4)

y_train = y_train.squeeze()
y_test = y_test.squeeze()

## Configure the hyperparameters

In [82]:
weight_decay = 0.0001
batch_size = 128
num_epochs = 50
dropout_rate = 0.2
image_size = 64  # We'll resize input images to this size.
patch_size = 8  # Size of the patches to be extracted from the input images.
num_patches = (image_size // patch_size) ** 2  # Size of the data array.
embedding_dim = 256  # Number of hidden units.
num_blocks = 4  # Number of blocks.

print(f"Image size: {image_size} X {image_size} = {image_size ** 2}")
print(f"Patch size: {patch_size} X {patch_size} = {patch_size ** 2} ")
print(f"Patches per image: {num_patches}")
print(f"Elements per patch (3 channels): {(patch_size ** 2) * 3}")

Image size: 64 X 64 = 4096
Patch size: 8 X 8 = 64 
Patches per image: 64
Elements per patch (3 channels): 192


## Build a classification model

We implement a method that builds a classifier given the processing blocks.

In [83]:
input_shape=(128,128,3)
num_classes=4
def build_classifier(blocks, positional_encoding=False):
    inputs = layers.Input(shape=input_shape)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size, num_patches)(augmented)
    # Encode patches to generate a [batch_size, num_patches, embedding_dim] tensor.
    x = layers.Dense(units=embedding_dim)(patches)
    if positional_encoding:
        positions = tf.range(start=0, limit=num_patches, delta=1)
        position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=embedding_dim
        )(positions)
        x = x + position_embedding
    # Process x using the module blocks.
    x = blocks(x)
    # Apply global average pooling to generate a [batch_size, embedding_dim] representation tensor.
    representation = layers.GlobalAveragePooling1D()(x)
    # Apply dropout.
    representation = layers.Dropout(rate=dropout_rate)(representation)
    # Compute logits outputs.
    logits = layers.Dense(num_classes)(representation)
    # Create the Keras model.
    return keras.Model(inputs=inputs, outputs=logits)


## Define an experiment

We implement a utility function to compile, train, and evaluate a given model.

In [84]:

def run_experiment(model):
    # Create Adam optimizer with weight decay.
    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay,
    )
    # Compile the model.
    model.compile(
        optimizer=optimizer,
        loss="categorical_crossentropy", # use categorical crossentropy loss
        metrics=["accuracy"],
    )
    # Create a learning rate scheduler callback.
    reduce_lr = keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=5
    )
    # Create an early stopping callback.
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10, restore_best_weights=True
    )
    # Fit the model.
    history = model.fit(
        new_x_train,
        new_y_train,
        batch_size=batch_size,
        epochs=50,
        validation_split=0.1,
        callbacks=[early_stopping, reduce_lr],
    )

    # Evaluate the model on the test set.
    y_pred = model.predict(x_test)
    accuracy = np.mean(np.argmax(y_test, axis=1) == np.argmax(y_pred, axis=1))
    top_5_accuracy = np.mean(
       [np.argmax(y_test[i]) in np.argsort(y_pred[i])[::-1][:5] for i in range(len(y_test))]
    )

# Print the test accuracies.
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return history


## Use data augmentation

In [85]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(x_train)


## Implement patch extraction as a layer

In [86]:

class Patches(layers.Layer):
    def __init__(self, patch_size, num_patches):
        super().__init__()
        self.patch_size = patch_size
        self.num_patches = num_patches

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, self.num_patches, patch_dims])
        return patches


## The MLP-Mixer model

The MLP-Mixer is an architecture based exclusively on
multi-layer perceptrons (MLPs), that contains two types of MLP layers:

1. One applied independently to image patches, which mixes the per-location features.
2. The other applied across patches (along channels), which mixes spatial information.

This is similar to a [depthwise separable convolution based model](https://arxiv.org/pdf/1610.02357.pdf)
such as the Xception model, but with two chained dense transforms, no max pooling, and layer normalization
instead of batch normalization.

### Implement the MLP-Mixer module

In [87]:

class MLPMixerLayer(layers.Layer):
    def __init__(self, num_patches, hidden_units, dropout_rate, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.mlp1 = keras.Sequential(
            [
                layers.Dense(units=num_patches),
                tfa.layers.GELU(),
                layers.Dense(units=num_patches),
                layers.Dropout(rate=dropout_rate),
            ]
        )
        self.mlp2 = keras.Sequential(
            [
                layers.Dense(units=num_patches),
                tfa.layers.GELU(),
                layers.Dense(units=embedding_dim),
                layers.Dropout(rate=dropout_rate),
            ]
        )
        self.normalize = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        # Apply layer normalization.
        x = self.normalize(inputs)
        # Transpose inputs from [num_batches, num_patches, hidden_units] to [num_batches, hidden_units, num_patches].
        x_channels = tf.linalg.matrix_transpose(x)
        # Apply mlp1 on each channel independently.
        mlp1_outputs = self.mlp1(x_channels)
        # Transpose mlp1_outputs from [num_batches, hidden_dim, num_patches] to [num_batches, num_patches, hidden_units].
        mlp1_outputs = tf.linalg.matrix_transpose(mlp1_outputs)
        # Add skip connection.
        x = mlp1_outputs + inputs
        # Apply layer normalization.
        x_patches = self.normalize(x)
        # Apply mlp2 on each patch independtenly.
        mlp2_outputs = self.mlp2(x_patches)
        # Add skip connection.
        x = x + mlp2_outputs
        return x


### Build, train, and evaluate the MLP-Mixer model

Note that training the model with the current settings on a V100 GPUs
takes around 8 seconds per epoch.

In [88]:
mlpmixer_blocks = keras.Sequential(
    [MLPMixerLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)]
)
learning_rate = 0.005
mlpmixer_classifier = build_classifier(mlpmixer_blocks)
history = run_experiment(mlpmixer_classifier)

Epoch 1/50
1/1 [==============================] - 7s 7s/step - loss: 8.9562 - accuracy: 0.5000 - val_loss: 10.8768 - val_accuracy: 0.6667 - lr: 0.0050
Epoch 2/50
1/1 [==============================] - 0s 78ms/step - loss: 10.2069 - accuracy: 0.6154 - val_loss: 10.8047 - val_accuracy: 0.6667 - lr: 0.0050
Epoch 3/50
1/1 [==============================] - 0s 76ms/step - loss: 10.0641 - accuracy: 0.6154 - val_loss: 10.7454 - val_accuracy: 0.6667 - lr: 0.0050
Epoch 4/50
1/1 [==============================] - 0s 62ms/step - loss: 9.9188 - accuracy: 0.6154 - val_loss: 10.7454 - val_accuracy: 0.6667 - lr: 0.0050
Epoch 5/50
1/1 [==============================] - 0s 89ms/step - loss: 8.0590 - accuracy: 0.6154 - val_loss: 5.3727 - val_accuracy: 0.6667 - lr: 0.0050
Epoch 6/50
1/1 [==============================] - 0s 65ms/step - loss: 4.9594 - accuracy: 0.6154 - val_loss: 5.3727 - val_accuracy: 0.6667 - lr: 0.0050
Epoch 7/50
1/1 [==============================] - 0s 64ms/step - loss: 4.9594 - accu

1/1 [==============================] - 2s 2s/step
Test accuracy: 0.0%
Test top 5 accuracy: 100.0%


The MLP-Mixer model tends to have much less number of parameters compared
to convolutional and transformer-based models, which leads to less training and
serving computational cost.

As mentioned in the [MLP-Mixer](https://arxiv.org/abs/2105.01601) paper,
when pre-trained on large datasets, or with modern regularization schemes,
the MLP-Mixer attains competitive scores to state-of-the-art models.
You can obtain better results by increasing the embedding dimensions,
increasing the number of mixer blocks, and training the model for longer.
You may also try to increase the size of the input images and use different patch sizes.

## The FNet model

The FNet uses a similar block to the Transformer block. However, FNet replaces the self-attention layer
in the Transformer block with a parameter-free 2D Fourier transformation layer:

1. One 1D Fourier Transform is applied along the patches.
2. One 1D Fourier Transform is applied along the channels.

### Implement the FNet module

In [91]:

class FNetLayer(layers.Layer):
    def __init__(self, num_patches, embedding_dim, dropout_rate, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.ffn = keras.Sequential(
            [
                layers.Dense(units=embedding_dim),
                tfa.layers.GELU(),
                layers.Dropout(rate=dropout_rate),
                layers.Dense(units=embedding_dim),
            ]
        )

        self.normalize1 = layers.LayerNormalization(epsilon=1e-6)
        self.normalize2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        # Apply fourier transformations.
        x = tf.cast(
            tf.signal.fft2d(tf.cast(inputs, dtype=tf.dtypes.complex64)),
            dtype=tf.dtypes.float32,
        )
        # Add skip connection.
        x = x + inputs
        # Apply layer normalization.
        x = self.normalize1(x)
        # Apply Feedfowrad network.
        x_ffn = self.ffn(x)
        # Add skip connection.
        x = x + x_ffn
        # Apply layer normalization.
        return self.normalize2(x)


### Build, train, and evaluate the FNet model

Note that training the model with the current settings on a V100 GPUs
takes around 8 seconds per epoch.

In [92]:
fnet_blocks = keras.Sequential(
    [FNetLayer(num_patches, embedding_dim, dropout_rate) for _ in range(num_blocks)]
)
learning_rate = 0.001
fnet_classifier = build_classifier(fnet_blocks, positional_encoding=True)
history = run_experiment(fnet_classifier)

Epoch 1/50


1/1 [==============================] - ETA: 0s - loss: 3.9614 - accuracy: 0.0769

1/1 [==============================] - 4s 4s/step - loss: 3.9614 - accuracy: 0.0769 - val_loss: 16.1181 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/50
1/1 [==============================] - 0s 69ms/step - loss: 11.8553 - accuracy: 0.0000e+00 - val_loss: 16.1181 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/50
1/1 [==============================] - 0s 72ms/step - loss: 10.5416 - accuracy: 0.0000e+00 - val_loss: 5.3727 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/50
1/1 [==============================] - 0s 70ms/step - loss: 6.8192 - accuracy: 0.0000e+00 - val_loss: 1.1921e-07 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 5/50
1/1 [==============================] - 0s 59ms/step - loss: 5.5793 - accuracy: 0.0000e+00 - val_loss: 1.1921e-07 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 6/50
1/1 [==============================] - 0s 68ms/step - loss: 3.7196 - accuracy: 0.0000e+00 - val_loss: 1.1921e-07 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 7/50
1/1 [=========================

1/1 [==============================] - 0s 484ms/step
Test accuracy: 0.0%
Test top 5 accuracy: 100.0%
